# Tree Dataset
## Cleaning and Shaping


In [1]:
import pandas as pd
import os
import re

TREATMENTS = {2: 'Control', 4: 'Control', 6: 'Control', 11: 'Control',
              3: 'T2: Radial', 5: 'T2: Radial', 7: 'T2: Radial', 9: 'T2: Radial',
              1: 'T1: Gap', 8: 'T1: Gap',10: 'T1: Gap', 12: 'T1: Gap'}

print('Initialised!')

Initialised!


## Let's load your data from the spreadsheets into this environment:

In [8]:
path = './excel/'
extensions = os.listdir(path)
paths = [path + e for e in extensions]

print("It looks like we're working with the following files:\n")
for p in paths:
  print('\t', p)

It looks like we're working with the following files:

	 ./excel/Plot 08.xlsx
	 ./excel/Plot 04.xlsx
	 ./excel/Plot 12.xlsx
	 ./excel/Plot 05.xlsx
	 ./excel/Plot 09.xlsx
	 ./excel/Plot 02.xlsx
	 ./excel/Plot 03.xlsx
	 ./excel/Plot 01.xlsx
	 ./excel/Plot 10.xlsx
	 ./excel/Plot 06.xlsx
	 ./excel/Plot 07.xlsx
	 ./excel/Plot 11.xlsx


In [9]:
master = pd.DataFrame(None)

for p in paths:
  df = pd.read_excel(p)
  df = df.iloc[:,0:9]

  plot = re.findall(r'\d+', p)[0]
  df['Plot Number'] = int(plot)

  master = pd.concat([master, df])
  master.index = range(0, len(master))

print('Joined all the files together!')



years = pd.unique(master['Year'])
print("It looks like we've got data from the following years:\n")
for y in years:
  print("Year", y)

Joined all the files together!
It looks like we've got data from the following years:

Year 0
Year 6


## Now, let's stack them all into a single table

In [10]:
cols = ['Tree Number', 'Plot', 'Treatment', 'Large']
for y in years:
  y = str(y)
  cols.append('Alive at year ' + y)
  cols.append('DBH year ' + str(y))
  cols.append('Hollows Class 1 year ' + y)
  cols.append('Hollows Class 2 year ' + y)
  cols.append('Hollows Class 3 year ' + y)
  
  master_matrix = pd.DataFrame(None, index=pd.unique(master['Tree Number']),
                                   columns=cols)

print('Created a template for the final table!')

Created a template for the final table!


In [11]:
for i in master.index:
  row = master.loc[i,:]

  if type(row['Alive']) == str:
    if row['Alive'][0].lower() == 'y': row['Alive'] = True
    elif row['Alive'][0].lower() == 'n': row['Alive'] = False

  if type(row['Large?']) == str:
    if row['Large?'][0].lower() == 'y': row['Large?'] = True
    elif row['Large?'][0].lower() == 'n': row['Large?'] = False

  for j in ['1','2','3']:
    h = row['Hollows Class ' + j]
    if type(h) == str:
      if h[0].lower() == 'y': row['Hollows Class ' + j] = 1
      else: row['Hollows Class ' + j] = 0

  y = str(row['Year'])
  tree = row['Tree Number']

  master_matrix.loc[tree, 'Tree Number'] = row['Tree Number']
  master_matrix.loc[tree, 'Plot'] = row['Plot Number']
  master_matrix.loc[tree, 'Treatment'] = TREATMENTS[row['Plot Number']]

  master_matrix.loc[tree, 'DBH year ' + y] = row['DBH']
  master_matrix.loc[tree, 'Hollows Class 1 year ' + y] = row['Hollows Class 1']
  master_matrix.loc[tree, 'Hollows Class 2 year ' + y] = row['Hollows Class 2']
  master_matrix.loc[tree, 'Hollows Class 3 year ' + y] = row['Hollows Class 3']

  if y == '0':
    master_matrix.loc[tree, 'Large'] = row['Large?']
    master_matrix.loc[tree, 'Alive at year ' + y] = True

  else:
    master_matrix.loc[tree, 'Alive at year ' + y] = row['Alive']

print('Loaded all data into the final table!')

/var/folders/w9/stmw5z4s3mq6qwwy8y59j9y40000gn/T/ipykernel_78021/2782417830.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if row['Large?'][0].lower() == 'y': row['Large?'] = True
/var/folders/w9/stmw5z4s3mq6qwwy8y59j9y40000gn/T/ipykernel_78021/2782417830.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif row['Large?'][0].lower() == 'n': row['Large?'] = False
/var/folders/w9/stmw5z4s3mq6qwwy8y59j9y40000gn/T/ipykernel_78021/2782417830.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

Loaded all data into the final table!


## Let's look at the first couple of rows from the final table:

In [12]:
master_matrix.head()

,Tree Number,Plot,Treatment,Large,Alive at year 0,DBH year 0,Hollows Class 1 year 0,Hollows Class 2 year 0,Hollows Class 3 year 0,Alive at year 6,DBH year 6,Hollows Class 1 year 6,Hollows Class 2 year 6,Hollows Class 3 year 6
60,60,8,T1: Gap,True,True,42.9,0.0,0.0,0.0,True,45.2,0.0,0.0,0.0
59,59,8,T1: Gap,True,True,44.0,0.0,0.0,0.0,True,48.9,0.0,0.0,0.0
54,54,8,T1: Gap,True,True,43.6,0.0,0.0,0.0,True,44.2,0.0,0.0,0.0
55,55,8,T1: Gap,True,True,50.2,0.0,0.0,0.0,True,53.3,0.0,0.0,0.0
244,244,8,T1: Gap,True,True,52.9,0.0,0.0,0.0,True,54.9,1.0,0.0,0.0


## Now let's export this table to a csv file:

In [11]:
output_path = './outputs/'
master_matrix.to_csv(output_path + 'tree_matrix.csv', index=False)

Give this a minute to run, then you should be able to download the file from the 'output' directory. 